In [30]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta
import requests

In [4]:
d1 = date(2016, 1, 1)  # start date
d2 = date(2016, 12, 31)  # end date

delta = d2 - d1  # timedelta

dates = []

In [21]:
#dates
dates = []

In [22]:
for i in range(delta.days + 1):
    dates.append((d1 + timedelta(days=i)).strftime("%Y/%#m/%#d"))

#print(dates)

In [27]:
urls = []
for i in dates:
        urls.append("https://www.wunderground.com/history/airport/KJFK/" + i + "/DailyHistory.html")

In [ ]:
urls

In [32]:
#for i in urls:
 #   soup = BeautifulSoup(i, 'lxml') # Parse the HTML as a string
  #  obs_table = soup.find_all('observations_details') # Grab the first table
    
url = urls[1]
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, "lxml")

In [36]:
soup.find_all('table')

[<table cellpadding="0" cellspacing="0" class="responsive airport-history-summary-table" id="historyTable">
 <thead>
 <tr>
 <th> </th>
 <th>Actual</th>
 <th>Average </th>
 <th>Record </th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td class="history-table-grey-header">Temperature</td>
 <td class="history-table-grey-header" colspan="3"> </td>
 </tr>
 <tr>
 <td class="indent"><span>Mean Temperature</span></td>
 <td>
 <span class="wx-data"><span class="wx-value">38</span><span class="wx-unit"> °F</span></span>
 </td>
 <td>
 <span class="wx-data"><span class="wx-value">34</span><span class="wx-unit"> °F</span></span>
 </td>
 <td> </td>
 </tr>
 <tr>
 <td class="indent"><span>Max Temperature</span></td>
 <td>
 <span class="wx-data"><span class="wx-value">41</span><span class="wx-unit"> °F</span></span>
 </td>
 <td>
 <span class="wx-data"><span class="wx-value">40</span><span class="wx-unit"> °F</span></span>
 </td>
 <td>
 <span class="wx-data"><span class="wx-value">57</span><span class="wx-unit"> °F